In [19]:
import numpy as np 
import pandas as pd
from nma import load_single_timeseries 
from nma import load_evs
from hdf5storage import savemat

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 100

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
RUNS   = ['LR','RL']
N_RUNS = 2

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond': ['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['mental','rnd']}}



my_exp = 'MOTOR'
subjects = np.loadtxt('subjects_list.txt', dtype='str')
nsubjects = len(subjects)
conditions = EXPERIMENTS[my_exp]['cond']
nconditions = len(conditions)
print(conditions)
subj_fmri = np.zeros((N_PARCELS,len(subjects)*len(conditions)))
condition_index = np.zeros(len(subjects)*len(conditions))
subject_index = np.zeros(len(subjects)*len(conditions))

for s in range(len(subjects)):
    my_subj = subjects[s]
    data = load_single_timeseries(subject=my_subj, experiment=my_exp, bold_run='RL')
    for j in range(data.shape[0]):
	    data[j,:] = data[j,:] - np.mean(data[j,:]) 
    evs = load_evs(subject=my_subj, experiment=my_exp, conditions = conditions,bold_run='RL')
    scan_index = dict()
    for j in range(len(evs)):
        x = evs[j]
        y = np.array(x[0])
        for k in range(1,len(x)):
            y = np.append(y,x[k])
        scan_index[conditions[j]] = y
    savg_RL = np.zeros((N_PARCELS,nconditions))
    for j in range(nconditions):
        savg_RL[:,j] = np.mean(data[:,scan_index[conditions[j]]],axis =1)
    data = load_single_timeseries(subject=my_subj,experiment=my_exp,bold_run='LR')
    for j in range(data.shape[0]):
	    data[j,:] = data[j,:] - np.mean(data[j,:])
    evs = load_evs(subject=my_subj, experiment=my_exp, conditions = conditions,bold_run='LR')
    scan_index = dict()
    for j in range(len(evs)):
        x = evs[j]
        y = np.array(x[0])
        for k in range(1,len(x)):
            y = np.append(y,x[k])
        scan_index[conditions[j]] = y
    savg_LR = np.zeros((N_PARCELS,nconditions))
    for j in range(nconditions):
        savg_LR[:,j] = np.mean(data[:,scan_index[conditions[j]]],axis =1)
    savg = (savg_LR + savg_RL)/2
    subj_fmri[:,s*nconditions:(s+1)*nconditions]= savg
    condition_index[s*nconditions:(s+1)*nconditions] = range(nconditions)
    subject_index[s*nconditions:(s+1)*nconditions] = s


#z_lf = np.mean(data[:,scan_index['lf']],axis = 1)
#z_rf = np.mean(data[:,scan_index['rf']],axis = 1)


['lf', 'rf', 'lh', 'rh', 't', 'cue']


In [20]:
print(scan_index[conditions[0]])


[ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178]


In [21]:
fmri = dict()
fmri['fmri'] = subj_fmri
fmri['subject'] = subject_index
fmri['condition_index'] = condition_index
fmri['conditions'] = conditions
fmri['task'] = my_exp
fmri['nsubjects'] = nsubjects
fmri['nconditions'] = nconditions
fmri['nregions'] = N_PARCELS

savemat(my_exp+'_fmri_subjectaverage.mat',fmri,store_python_metadata=True)